import libraries

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim
import torch.utils.data as du
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy
import os
import time
from joblib import load

read data

In [9]:
'''
train = os.listdir('train_dataset/')
valid = os.listdir('valid_dataset/')
test = os.listdir('test_dataset/')

train_x = load('train_dataset/' + 'shard-0-X.joblib') # (276216, 101, 1)
valid_x = load('valid_dataset/' + valid[6]) # (34527, 101, 4)
test_x = load('test_dataset/' + test[6]) # (34528, 101, 1)
'''

"\ntrain = os.listdir('train_dataset/')\nvalid = os.listdir('valid_dataset/')\ntest = os.listdir('test_dataset/')\n\ntrain_x = load('train_dataset/' + 'shard-0-X.joblib') # (276216, 101, 1)\nvalid_x = load('valid_dataset/' + valid[6]) # (34527, 101, 4)\ntest_x = load('test_dataset/' + test[6]) # (34528, 101, 1)\n"

In [10]:
class JUND_Dataset(Dataset):
    def __init__(self, data_dir):
        '''load X, y, w, a from data_dir'''        
        super(JUND_Dataset, self).__init__()

        # load X, y, w, a from given data_dir
        # convert them into torch tensors

        x = load(data_dir + 'shard-0-X.joblib')
        y = load(data_dir +'shard-0-y.joblib')
        w = load(data_dir + 'shard-0-w.joblib')
        # ids = load(data_dir + 'shard-0-ids.joblib')
        a = load(data_dir + 'shard-0-a.joblib')

        self.x = torch.tensor(x).float()  
        self.y = torch.tensor(y).float()
        self.w = torch.tensor(w).float()
        # self.ids = torch.tensor(ids)
        self.a = torch.tensor(a).float()
        
    def __len__(self):
        '''return len of dataset'''
        return len(self.y)
    
    def __getitem__(self, idx):
        '''return X, y, w, and a values at index idx'''
       
        _x = self.x[idx]
        _y = self.y[idx]
        _w = self.w[idx]
        # _ids = self.ids[idx]
        _a = self.a[idx]
        
        return _x, _y, _w, _a

define MLP model

In [11]:
class MLP(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        '''in_dim: input layer dim
           hidden_dim: hidden layer dim
           out_dim: output layer dim'''
        
        super(MLP, self).__init__()
        
        # images are 28x28 so flatten them into 784d vec
        self.flatten = nn.Flatten()
        
        #two fully connected layers
        self.fc1 = nn.Linear(in_dim, hidden_dim-1)
        self.fc2 = nn.Linear(hidden_dim, out_dim)

    def forward(self, x, a):
        # since x is 28x28, flatten it first
        x = self.flatten(x)
        
        # compute output of fc1, and apply relu activation
        x = functional.relu(self.fc1(x))
        
        # compute output layer
        # no activation
        x = torch.cat((x, a), dim=1)
        x = self.fc2(x)
        return x

Set up training

In [12]:
device = f'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"using device: {device}")
batch_size = 1000
learning_rate = 0.5 #0.01
epochs = 30

# set model and optimizer
# 104*4 as inputs
# use 128 hidden layer + 1 accessibility value node
# output is binary so 1
model = MLP(101*4, 129, 1)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# load training data in batches
train_loader = du.DataLoader(dataset=JUND_Dataset('train_dataset/'),
                             batch_size=batch_size,
                             shuffle=True)
# send model over to device
model = model.to(device)
model.train()

using device: cuda:0


MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=404, out_features=128, bias=True)
  (fc2): Linear(in_features=129, out_features=1, bias=True)
)

Training loop over batches

In [14]:
start = time.time()

for epoch in range(1, epochs + 1):    
    sum_loss = 0.
    
    for batch_idx, data in enumerate(train_loader):
        # send batch over to device
        x, y, w, a = data
        x, y, a, w = x.to(device), y.to(device), a.to(device), w.to(device)

        # zero out prev gradients
        optimizer.zero_grad()
        
        # run the forward pass
        output = model(x, a)
        
        # compute loss/error
        loss = functional.binary_cross_entropy_with_logits(output, y, weight=w)
   
            
        # sum up batch losses
        sum_loss += loss.item()
        
        # compute gradients and take a step
        loss.backward()
        optimizer.step()
    
    # average loss per example    
    sum_loss /= len(train_loader.dataset)
    print(f'Epoch: {epoch}, Loss: {sum_loss:.6f}')

end = time.time()
print("time: ", end-start)

Epoch: 1, Loss: 0.000536
Epoch: 2, Loss: 0.000535
Epoch: 3, Loss: 0.000538
Epoch: 4, Loss: 0.000538
Epoch: 5, Loss: 0.000536
Epoch: 6, Loss: 0.000536
Epoch: 7, Loss: 0.000532
Epoch: 8, Loss: 0.000533
Epoch: 9, Loss: 0.000538
Epoch: 10, Loss: 0.000537
Epoch: 11, Loss: 0.000535
Epoch: 12, Loss: 0.000535
Epoch: 13, Loss: 0.000538
Epoch: 14, Loss: 0.000536
Epoch: 15, Loss: 0.000533
Epoch: 16, Loss: 0.000537
Epoch: 17, Loss: 0.000541
Epoch: 18, Loss: 0.000532
Epoch: 19, Loss: 0.000533
Epoch: 20, Loss: 0.000535
Epoch: 21, Loss: 0.000534
Epoch: 22, Loss: 0.000543
Epoch: 23, Loss: 0.000535
Epoch: 24, Loss: 0.000541
Epoch: 25, Loss: 0.000532
Epoch: 26, Loss: 0.000540
Epoch: 27, Loss: 0.000536
Epoch: 28, Loss: 0.000530
Epoch: 29, Loss: 0.000541
Epoch: 30, Loss: 0.000533
time:  96.86818504333496


Testing

In [15]:
# load test images in batches
test_loader = du.DataLoader(dataset=JUND_Dataset('test_dataset/'),
                            batch_size=batch_size,
                            shuffle=True)

# set model in eval mode, since we are no longer training
model.eval()
test_loss = 0
correct = 0

# turn of gradient computation, will speed up testing
with torch.no_grad():
    for batch_idx, data in enumerate(test_loader):
        # send batches to device
        x, y, w, a = data
        x, y, a, w = x.to(device), y.to(device), a.to(device), w.to(device)
        
        # compute forward pass
        output = model(x, a)

        # compute loss/error
        loss = functional.binary_cross_entropy_with_logits(output, y, weight=w)
        
        # sum up batch loss
        test_loss += loss.item()

        # get the index/class of the max log-probability
        # output = F.log_softmax(output, dim=1)
        # pred = output.max(dim=1)[1]

        m = nn.Sigmoid()
        output = m(output)
        
        output = torch.where(output<0.5, output, torch.tensor(1.).to(device))
        output = torch.where(output==1.0, output, torch.tensor(0.).to(device))

        # add up number of correct predictions
        correct += torch.sum(output == y)
  
    # test loss per example
    test_loss /= len(test_loader.dataset)
    
    # final test accuracy
    test_acc = correct / len(test_loader.dataset)
    print(f'Test loss: {test_loss:.6f}, accuracy: {test_acc:.4f}',
          f'correct: {correct}')

Test loss: 0.000528, accuracy: 0.9044 correct: 31227
